In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/siamese-BERT-fake-news-detection-LIAR/")

In [3]:
import pandas as pd
import numpy as np

In [4]:

train_path = './LIAR-PLUS/dataset/train2.tsv'
test_path = './LIAR-PLUS/dataset/test2.tsv'
val_path = './LIAR-PLUS/dataset/val2.tsv'


In [5]:
train_df = pd.read_csv(train_path, sep="\t", header=None)
test_df = pd.read_csv(test_path, sep="\t", header=None)
val_df = pd.read_csv(val_path, sep="\t", header=None)

In [6]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
val_df = val_df.fillna(0)

In [7]:
# Define content and metadata components
renamed_columns = {
    0: "index",
    1: "statement_id",
    2: "label",
    3: "statement",
    4: "subject",
    5: "speaker",
    6: "speaker_job_title",
    7: "state_info",
    8: "party_affiliation",
    9: "barely_true_counts",
    10: "false_counts",
    11: "half_true_counts",
    12: "mostly_true_counts",
    13: "pants_on_fire_counts",
    14: "context",
    15: "justification"
}
train_df = train_df.rename(columns=renamed_columns)
val_df = val_df.rename(columns=renamed_columns)
test_df = test_df.rename(columns=renamed_columns)

In [8]:

label_map = {
    "true": 0,
    "mostly-true": 0,
    "half-true": 0,
    "barely-true": 1,
    "false": 1,
    "pants-fire": 1
}
to_binary_label = lambda label: label_map.get(label, 0)

In [9]:
y_train = train_df['label'].apply(to_binary_label).values
y_val = val_df['label'].apply(to_binary_label).values
y_test = test_df['label'].apply(to_binary_label).values

In [10]:
X_train_statement = train_df['statement'].values
X_val_statement = val_df['statement'].values
X_test_statement = test_df['statement'].values

In [11]:
to_metadata = lambda record: f"{record['subject']} {record['speaker']} {record['speaker_job_title']} {record['state_info']} {record['party_affiliation']} {record['context']}"

In [12]:
X_train_metadata = train_df.apply(to_metadata, axis=1).values
X_val_metadata = val_df.apply(to_metadata, axis=1).values
X_test_metadata = test_df.apply(to_metadata, axis=1).values

In [13]:
# !pip install pytorch-pretrained-bert transformers
! pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.4 MB/s eta 0:00:00


In [17]:
import torch
from torch import nn
# from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import time
from tqdm import tqdm
from transformers import RobertaModel, RobertaTokenizer
# from transformers import XLMRobertaTokenizer, XLMRobertaModel
from sklearn.metrics import accuracy_score, f1_score
import copy



In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [19]:
# model_name = 'xlm-roberta-base'
# num_labels = 2 # 2 classes for binary classification
# bert_model = XLMRobertaModel.from_pretrained(model_name)
# tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
bert_model = RobertaModel.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

for param in bert_model.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
train_statement_encoding = tokenizer(list(X_train_statement), truncation=True, padding=True, return_tensors='pt', return_length=True)
train_metadata_encoding = tokenizer(list(X_train_metadata), truncation=True, padding=True, return_tensors='pt', return_length=True)
val_statement_encoding = tokenizer(list(X_val_statement), truncation=True, padding=True, return_tensors='pt', return_length=True)
val_metadata_encoding = tokenizer(list(X_val_metadata), truncation=True, padding=True, return_tensors='pt', return_length=True)
test_statement_encoding = tokenizer(list(X_test_statement), truncation=True, padding=True, return_tensors='pt', return_length=True)
test_metadata_encoding = tokenizer(list(X_test_metadata), truncation=True, padding=True, return_tensors='pt', return_length=True)


In [18]:
train_encodings = torch.utils.data.TensorDataset(
    train_statement_encoding['input_ids'],
    train_statement_encoding['attention_mask'],
    train_metadata_encoding['input_ids'],
    train_metadata_encoding['attention_mask'],
    torch.tensor(y_train)
)

val_encodings = torch.utils.data.TensorDataset(
    val_statement_encoding['input_ids'],
    val_statement_encoding['attention_mask'],
    val_metadata_encoding['input_ids'],
    val_metadata_encoding['attention_mask'],
    torch.tensor(y_val)
)

test_encodings = torch.utils.data.TensorDataset(
    test_statement_encoding['input_ids'],
    test_statement_encoding['attention_mask'],
    test_metadata_encoding['input_ids'],
    test_metadata_encoding['attention_mask'],
    torch.tensor(y_test)
)

train_encoding_loader = DataLoader(train_encodings, batch_size=16, shuffle=False)
val_encoding_loader = DataLoader(val_encodings, batch_size=16, shuffle=False)
test_encoding_loader = DataLoader(test_encodings, batch_size=16, shuffle=False)

In [19]:
bert_model.to(device)

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [20]:
with torch.no_grad():
  train_embeddings = []
  for batch in tqdm(train_encoding_loader):
    statement_input_ids = batch[0].to(device)
    statement_attention_mask = batch[1].to(device)
    # metadata_input_ids = batch[2].to(device)
    # metadata_attention_mask = batch[3].to(device)
    # labels = batch[4].to(device)

    statement_embedding = bert_model(statement_input_ids, attention_mask=statement_attention_mask)
    statement_embedding = statement_embedding.last_hidden_state[:, 0, :] #using the CLS embedding
    # statement_embedding = torch.mean(statement_embedding.last_hidden_state, dim=1)
    
    # metadata_embedding = bert_model(metadata_input_ids, attention_mask=metadata_attention_mask)
    # # metadata_embedding = metadata_embedding.last_hidden_state[:, 0, :]
    # metadata_embedding = torch.mean(metadata_embedding.last_hidden_state, dim=1)

    # input = torch.cat([statement_embedding, metadata_embedding], dim=-1)
    input = statement_embedding
    train_embeddings.append(input.cpu())

  train_embeddings = torch.cat(train_embeddings, dim=0)

100%|██████████| 640/640 [04:51<00:00,  2.19it/s]


In [22]:
# np.save('train_embeddings.npy', train_embeddings.cpu().numpy())
# np.save('train_embeddings_mean.npy', train_embeddings.cpu().numpy())
np.save('train_embeddings_statement_CLS.npy', train_embeddings.cpu().numpy())

In [23]:
train_embeddings.shape

torch.Size([10240, 768])

In [28]:
with torch.no_grad():
  val_embeddings = []
  for batch in tqdm(val_encoding_loader):
    statement_input_ids = batch[0].to(device)
    statement_attention_mask = batch[1].to(device)
    # metadata_input_ids = batch[2].to(device)
    # metadata_attention_mask = batch[3].to(device)
    # labels = batch[4].to(device)

    statement_embedding = bert_model(statement_input_ids, attention_mask=statement_attention_mask)
    statement_embedding = statement_embedding.last_hidden_state[:, 0, :] #using the CLS embedding
    # statement_embedding = torch.mean(statement_embedding.last_hidden_state, dim=1)
    
    # metadata_embedding = bert_model(metadata_input_ids, attention_mask=metadata_attention_mask)
    # # metadata_embedding = metadata_embedding.last_hidden_state[:, 0, :]
    # metadata_embedding = torch.mean(metadata_embedding.last_hidden_state, dim=1)

    # input = torch.cat([statement_embedding, metadata_embedding], dim=-1)
    input = statement_embedding
    val_embeddings.append(input.cpu())

  val_embeddings = torch.cat(val_embeddings, dim=0)

100%|██████████| 81/81 [00:05<00:00, 14.39it/s]


In [29]:
# np.save('val_embeddings.npy', val_embeddings.cpu().numpy())
# np.save('val_embeddings_mean.npy', val_embeddings.cpu().numpy())
np.save('val_embeddings_statement_CLS.npy', val_embeddings.cpu().numpy())

In [30]:
val_embeddings.shape

torch.Size([1284, 768])

In [24]:
with torch.no_grad():
  test_embeddings = []
  for batch in tqdm(test_encoding_loader):
    statement_input_ids = batch[0].to(device)
    statement_attention_mask = batch[1].to(device)
    # metadata_input_ids = batch[2].to(device)
    # metadata_attention_mask = batch[3].to(device)
    # labels = batch[4].to(device)

    statement_embedding = bert_model(statement_input_ids, attention_mask=statement_attention_mask)
    statement_embedding = statement_embedding.last_hidden_state[:, 0, :] #using the CLS embedding
    # statement_embedding = torch.mean(statement_embedding.last_hidden_state, dim=1)

    # metadata_embedding = bert_model(metadata_input_ids, attention_mask=metadata_attention_mask)
    # # metadata_embedding = metadata_embedding.last_hidden_state[:, 0, :]
    # metadata_embedding = torch.mean(metadata_embedding.last_hidden_state, dim=1)

    # input = torch.cat([statement_embedding, metadata_embedding], dim=-1)
    input = statement_embedding
    test_embeddings.append(input.cpu())

  test_embeddings = torch.cat(test_embeddings, dim=0)

100%|██████████| 80/80 [00:36<00:00,  2.20it/s]


In [26]:
# np.save('test_embeddings.npy', test_embeddings.cpu().numpy())
# np.save('test_embeddings_mean.npy', test_embeddings.cpu().numpy())
np.save('test_embeddings_statement_CLS.npy', test_embeddings.cpu().numpy())

In [25]:
test_embeddings.shape

torch.Size([1267, 768])

In [24]:
# train_embeddings = torch.from_numpy(np.load('train_embeddings.npy'))
# val_embeddings = torch.from_numpy(np.load('val_embeddings.npy'))
# test_embeddings = torch.from_numpy(np.load('test_embeddings.npy'))

# train_embeddings = torch.from_numpy(np.load('train_embeddings_mean.npy'))
# val_embeddings = torch.from_numpy(np.load('val_embeddings_mean.npy'))
# test_embeddings = torch.from_numpy(np.load('test_embeddings_mean.npy'))

train_embeddings = torch.from_numpy(np.load('train_embeddings_statement_CLS.npy'))
val_embeddings = torch.from_numpy(np.load('val_embeddings_statement_CLS.npy'))
test_embeddings = torch.from_numpy(np.load('test_embeddings_statement_CLS.npy'))

In [25]:
train_dataset = torch.utils.data.TensorDataset(
    train_embeddings,
    torch.tensor(y_train)
)

val_dataset = torch.utils.data.TensorDataset(
    val_embeddings,
    torch.tensor(y_val)
)

test_dataset = torch.utils.data.TensorDataset(
    test_embeddings,
    torch.tensor(y_test)
)

In [26]:
# train_dataset = torch.utils.data.TensorDataset(
#     torch.tensor(train_statement_encoding['input_ids']),
#     torch.tensor(train_statement_encoding['attention_mask']),
#     torch.tensor(train_metadata_encoding['input_ids']),
#     torch.tensor(train_metadata_encoding['attention_mask']),
#     torch.tensor(y_train)
# )

# val_dataset = torch.utils.data.TensorDataset(
#     torch.tensor(val_statement_encoding['input_ids']),
#     torch.tensor(val_statement_encoding['attention_mask']),
#     torch.tensor(val_metadata_encoding['input_ids']),
#     torch.tensor(val_metadata_encoding['attention_mask']),
#     torch.tensor(y_val)
# )


In [27]:
# train_dataset = torch.utils.data.TensorDataset(
#     torch.tensor(train_statement_encoding['input_ids']),
#     torch.tensor(train_statement_encoding['attention_mask']),
#     torch.tensor(train_metadata_encoding['input_ids']),
#     torch.tensor(train_metadata_encoding['attention_mask']),
#     torch.tensor(y_train)
# )

# val_dataset = torch.utils.data.TensorDataset(
#     torch.tensor(val_statement_encoding['input_ids']),
#     torch.tensor(val_statement_encoding['attention_mask']),
#     torch.tensor(val_metadata_encoding['input_ids']),
#     torch.tensor(val_metadata_encoding['attention_mask']),
#     torch.tensor(y_val)
# )

# metadata_train_dataset = torch.utils.data.TensorDataset(
#     torch.tensor(train_metadata_encoding['input_ids']),
#     torch.tensor(train_metadata_encoding['attention_mask']),
#     torch.tensor(y_train)
# )

# metadata_val_dataset = torch.utils.data.TensorDataset(
#     torch.tensor(val_metadata_encoding['input_ids']),
#     torch.tensor(val_metadata_encoding['attention_mask']),
#     torch.tensor(y_val)
# )



In [28]:
# statement_train_loader = DataLoader(statement_train_dataset, batch_size=16, shuffle=True)
# statement_val_loader = DataLoader(statement_val_dataset, batch_size=16, shuffle=False)

# metadata_train_loader = DataLoader(metadata_train_dataset, batch_size=16, shuffle=True)
# metadata_val_loader = DataLoader(metadata_val_dataset, batch_size=16, shuffle=False)

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [30]:
# classifier = torch.nn.Sequential(
#     torch.nn.Linear(2 * 768, 256),
#     torch.nn.ReLU(),
#     torch.nn.Linear(256, 2)
# )
# classifier

classifier = torch.nn.Sequential(
    torch.nn.Linear(768, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 2)
)
classifier

Sequential(
  (0): Linear(in_features=768, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=2, bias=True)
)

In [31]:
# optimizer = torch.optim.Adam(classifier.parameters(), lr=2e-5, eps=1e-8)
# epochs = 200
# classifier = classifier.to(device)
# bert_model = bert_model.to(device)
# loss_fn = nn.CrossEntropyLoss()
# total_steps = len(train_loader) * epochs
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=5e-5, total_steps=total_steps)

In [32]:
# optimizer = torch.optim.Adam(classifier.parameters(), lr=0.0001, weight_decay=0.0001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
# epochs = 200
# classifier = classifier.to(device)
# bert_model = bert_model.to(device)
# loss_fn = nn.CrossEntropyLoss()
# total_steps = len(train_loader) * epochs
# # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=5e-5, total_steps=total_steps)

In [33]:
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)
epochs = 200
classifier = classifier.to(device)
bert_model = bert_model.to(device)
loss_fn = nn.CrossEntropyLoss()
total_steps = len(train_loader) * epochs

In [34]:

best_val_f1 = 0
best_test_f1 = 0
best_val_model = None
best_test_model = None

for epoch in range(epochs):

  print(f"Epoch {epoch + 1}/ {epochs}")
  print("-" * 50)
  # Train loop
  classifier.train()
  total_train_loss = 0

  for inputs, labels in tqdm(train_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_train_loss += current_loss.item()
    classifier.zero_grad()
    current_loss.backward()
    optimizer.step()
  # scheduler.step()

  avg_train_loss = total_train_loss / len(train_loader)

  classifier.eval()
  total_val_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_val_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_val_loss = total_val_loss / len(val_loader)
  val_acc = accuracy_score(true, pred)
  val_f1 = f1_score(true, pred)
  if val_f1 > best_val_f1:
    best_val_f1 = val_f1
    best_val_model = copy.deepcopy(classifier)

  classifier.eval()
  total_test_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_test_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_test_loss = total_test_loss / len(test_loader)
  test_acc = accuracy_score(true, pred)
  test_f1 = f1_score(true, pred)
  if test_f1 > best_test_f1:
    best_test_f1 = test_f1
    best_test_model = copy.deepcopy(classifier)
    
  print(f"Train loss: {avg_train_loss}, Val loss: {avg_val_loss}, Test loss: {avg_test_loss}")
  print(f"Val acc: {val_acc}, Test acc: {test_acc}")
  print(f"Val f1: {val_f1}, Test f1: {test_f1}")

print("Using CLS token Embedding")
print(f"Best val F1: {best_val_f1}")
print(f"Best test F1: {best_test_f1}")

Epoch 1/ 200
--------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 166.38it/s]


Train loss: 0.6850460723042489, Val loss: 0.6947578489780426, Test loss: 0.682330334186554
Val acc: 0.5202492211838006, Test acc: 0.56353591160221
Val f1: 0.0, Test f1: 0.0
Epoch 2/ 200
--------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 215.20it/s]


Train loss: 0.6814757362008095, Val loss: 0.695783664782842, Test loss: 0.6802343606948853
Val acc: 0.5202492211838006, Test acc: 0.56353591160221
Val f1: 0.0, Test f1: 0.0
Epoch 3/ 200
--------------------------------------------------


...

Train loss: 0.6255709677934647, Val loss: 0.6814490457375845, Test loss: 0.6514270544052124
Val acc: 0.6137071651090342, Test acc: 0.6101026045777427
Val f1: 0.5137254901960785, Test f1: 0.4535398230088495
Epoch 199/ 200
--------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 271.88it/s]


Train loss: 0.6254599586129188, Val loss: 0.6773635844389597, Test loss: 0.6502176880836487
Val acc: 0.6137071651090342, Test acc: 0.6148382004735596
Val f1: 0.5320754716981131, Test f1: 0.4841437632135307
Epoch 200/ 200
--------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 271.88it/s]


Train loss: 0.6249903574585914, Val loss: 0.6718280414740244, Test loss: 0.6498202443122864
Val acc: 0.6152647975077882, Test acc: 0.6148382004735596
Val f1: 0.5643738977072311, Test f1: 0.5215686274509803
Using CLS token Embedding
Best val F1: 0.619482496194825
Best test F1: 0.5952782462057337


In [103]:
torch.save(best_val_model.state_dict(), 'liar-bval-cls-embedding.pt')
torch.save(best_test_model.state_dict(), 'liar-btest-cls-embedding.pt')

In [113]:
best_val_f1 = 0
best_test_f1 = 0
best_val_model = None
best_test_model = None


for epoch in range(epochs):

  print(f"Epoch {epoch + 1}/ {epochs}")
  print("-" * 50)
  # Train loop
  classifier.train()
  total_train_loss = 0

  for inputs, labels in tqdm(train_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_train_loss += current_loss.item()
    classifier.zero_grad()
    current_loss.backward()
    optimizer.step()
  # scheduler.step()

  avg_train_loss = total_train_loss / len(train_loader)

  classifier.eval()
  total_val_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_val_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_val_loss = total_val_loss / len(val_loader)
  val_acc = accuracy_score(true, pred)
  val_f1 = f1_score(true, pred)

  if val_f1 > best_val_f1:
    best_val_f1 = val_f1
    best_val_model = copy.deepcopy(classifier)

  classifier.eval()
  total_test_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_test_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_test_loss = total_test_loss / len(test_loader)
  test_acc = accuracy_score(true, pred)
  test_f1 = f1_score(true, pred)

  if test_f1 > best_test_f1:
    best_test_f1 = test_f1
    best_test_model = copy.deepcopy(classifier)
    
  print(f"Train loss: {avg_train_loss}, Val loss: {avg_val_loss}, Test loss: {avg_test_loss}")
  print(f"Val acc: {val_acc}, Test acc: {test_acc}")
  print(f"Val f1: {val_f1}, Test f1: {test_f1}")



Epoch 1/ 200
--------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 268.99it/s]


Train loss: 0.6917014360427857, Val loss: 0.6880454222361246, Test loss: 0.6834417700767517
Val acc: 0.5202492211838006, Test acc: 0.5627466456195738
Val f1: 0.0, Test f1: 0.0
Epoch 2/ 200
--------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 359.82it/s]


Train loss: 0.6819199740886688, Val loss: 0.6867010394732157, Test loss: 0.6802854657173156
Val acc: 0.5202492211838006, Test acc: 0.5643251775848461
Val f1: 0.0, Test f1: 0.003610108303249097
Epoch 3/ 200
--------------------------------------------------


...

Train loss: 0.5844504594802856, Val loss: 0.6812388797601064, Test loss: 0.6460553288459778
Val acc: 0.6253894080996885, Test acc: 0.6282557221783741
Val f1: 0.5076765609007164, Test f1: 0.5698630136986301
Epoch 199/ 200
--------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 314.28it/s]


Train loss: 0.5848752990365028, Val loss: 0.6704104940096537, Test loss: 0.6527179956436158
Val acc: 0.6285046728971962, Test acc: 0.6274664561957379
Val f1: 0.5409047160731472, Test f1: 0.5888501742160278
Epoch 200/ 200
--------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 277.65it/s]

Train loss: 0.5842123627662659, Val loss: 0.6834574341773987, Test loss: 0.6489435791969299
Val acc: 0.6230529595015576, Test acc: 0.6298342541436464
Val f1: 0.49896480331262943, Test f1: 0.5653382761816497


In [114]:
print(f"Using Mean embedding")
print(f"Best val f1: {best_val_f1}")
print(f"Best test f1: {best_test_f1}")

Using Mean embedding
Best val f1: 0.595968448729185
Best test f1: 0.6161063330727131


In [115]:
torch.save(best_val_model.state_dict(), 'liar-bval-mean-embedding.pt')
torch.save(best_test_model.state_dict(), 'liar-btest-mean-embedding.pt')

```
LIAR Results

Using CLS token Embedding (Statement + Metadata)
Best val F1: 0.6477611940298508
Best test F1: 0.617363344051447

Using Mean embedding (Statement + Metadata)
Best val f1: 0.595968448729185
Best test f1: 0.6161063330727131


Using CLS token Embedding (Statement only)
Best val F1: 0.619482496194825
Best test F1: 0.5952782462057337
```

In [64]:
pred = torch.tensor([])
torch.cat([pred, torch.argmax(logits, dim=1)], 0)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [66]:


accuracy_score(labels, torch.cat([pred, torch.argmax(logits, dim=1)], 0))

0.5625

In [ ]:
for epoch in range(epochs):
    # Training loop
    classifier.train()
    total_train_loss = 0
    for batch in train_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        classifier.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        cls_embedding = last_hidden_state[:, 0, :]
        logits = classifier(cls_embedding)
        loss = torch.nn.functional.cross_entropy(logits, labels)
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(classifier.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_train_loss / len(train_loader)

    # Validation loop
    classifier.eval()
    total_val_loss = 0
    for batch in val_loader:
        input_ids = batch[0].to(device)
        attention





In [22]:
# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
# scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(statement_train_loader)*10)
model = model.to(device)

In [22]:
# for epoch in range(10):
#   print(f"Epoch {epoch+1}/ 10")
#   model.train()
#   train_loss = 0

#   for batch in tqdm(statement_train_loader):
#     batch = tuple(t.to(device) for t in batch)
#     inputs = {
#         'input_ids': batch[0],
#         'attention_mask': batch[1],
#         'labels': batch[2]
#     }
#     outputs = model(**inputs)
#     loss = outputs.loss
#     train_loss += loss.item()
#     optimizer.zero_grad()
#     optimizer.step()
#     scheduler.step()

#   model.eval()
#   val_loss, val_accuracy = 0, 0
#   for batch in tqdm(statement_val_loader):
#       batch = tuple(t.to(device) for t in batch)
#       with torch.no_grad():
#           inputs = {'input_ids': batch[0],
#                     'attention_mask': batch[1],
#                     'labels': batch[2]}
#           outputs = model(**inputs)
#           loss = outputs.loss
#           logits = outputs.logits
#       val_loss += loss.item()
#       preds = torch.argmax(logits, dim=1)
#       val_accuracy += torch.sum(preds == batch[2]).item()

#   # Print the training and validation loss and accuracy for each epoch
#   train_loss /= len(statement_train_loader)
#   val_loss /= len(statement_val_loader)
#   val_accuracy /= len(statement_val_dataset)
#   print(f'Epoch {epoch+1}: train_loss={train_loss:.4f} val_loss={val_loss:.4f} val_accuracy={val_accuracy:.4f}')
  

Epoch 1/ 10


  0%|          | 1/640 [00:02<28:42,  2.70s/it]


OutOfMemoryError: ignored

In [24]:
classifier = nn.Sequential(
    nn.Linear(768, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, 2)
)

# Set the device
classifier.to(device)

# Set the optimizer and learning rate scheduler
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
loss_fn = nn.CrossEntropyLoss().to(device)

In [31]:
val_accuracy = 0
for batch in tqdm(statement_val_loader):
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  }
        with torch.no_grad():
            outputs = model(**inputs)
            # features = outputs.last_hidden_state[:, 0, :]

        logits = outputs.logits
        labels = batch[2]
        # logits = classifier(features)
        # loss = loss_fn(logits, labels)
        # val_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        val_accuracy += torch.sum(preds == labels).item()

100%|██████████| 81/81 [05:40<00:00,  4.20s/it]


In [38]:
val_accuracy / len(statement_val_dataset)

0.5202492211838006

In [27]:
for epoch in range(10):
    classifier.train()
    train_loss = 0
    for batch in tqdm(statement_train_loader):
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1]}
        with torch.no_grad():
            outputs = model(**inputs)
            print(outputs)
            features = outputs.last_hidden_state[:, 0, :]
        labels = batch[2]
        logits = classifier(features)
        loss = loss_fn(logits, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    # Evaluate the fully-connected layer on the validation set
    classifier.eval()
    val_loss, val_accuracy = 0, 0
    for batch in tqdm(statement_val_loader):
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1]}
        with torch.no_grad():
            outputs = model(**inputs)
            features = outputs.last_hidden_state[:, 0, :]
        labels = batch[2]
        logits = classifier(features)
        loss = loss_fn(logits, labels)
        val_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        val_accuracy += torch.sum(preds == labels).item()

    # Print the training and validation loss and accuracy for each epoch
    train_loss /= len(statement_train_loader)
    val_loss /= len(statement_val_loader)
    val_accuracy /= len(statement_val_dataset)
    print(f'Epoch {epoch+1}: train_loss={train_loss:.4f} val_loss={val_loss:.4f} val_accuracy={val_accuracy:.4f}')

  0%|          | 0/640 [00:45<?, ?it/s]

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0452, -0.1555],
        [-0.0309, -0.1626],
        [-0.0455, -0.1679],
        [-0.0389, -0.1662],
        [-0.0485, -0.1567],
        [-0.0581, -0.1645],
        [-0.0528, -0.1566],
        [-0.0418, -0.1500],
        [-0.0372, -0.1697],
        [-0.0373, -0.1790],
        [-0.0362, -0.1664],
        [-0.0265, -0.1688],
        [-0.0427, -0.1565],
        [-0.0466, -0.1611],
        [-0.0351, -0.1607],
        [-0.0324, -0.1668]]), hidden_states=None, attentions=None)


AttributeError: ignored

In [ ]:
class custom_dataset(Dataset):
    def __init__(self, statement, metadata, label):
        self.statement = statement
        self.metadata = metadata
        self.label = label

    def __getitem__(self,index):
        statement_encodings = tokenizer(self.statement[index], truncation=True, padding=True)
        metadata_encodings = tokenizer(self.metadata[index], truncation=True, padding=True)
        
        tokenized_statement = tokenizer.tokenize(self.statement[index])
        statement_id = tokenizer.convert_tokens_to_ids(tokenized_statement)
        statement_id = statement_id[:min(len(statement_id), self.max_len_statement)]
        
        tokenized_metadata = tokenizer.tokenize(self.metadata[index])
        metadata_id = tokenizer.convert_tokens_to_ids(tokenized_metadata)
        metadata_id = metadata_id[:min(len(metadata_id), self.max_len_metadata)]

        label = self.label[index]

        return torch.tensor(statement_id), torch.tensor(metadata_id), label

    def __len__(self):
        return len(self.statement)

{'input_ids': [0, 104, 4113, 5, 3921, 918, 9527, 559, 333, 4548, 371, 12, 4328, 38417, 17600, 15, 1077, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 2623520.90B/s]


In [ ]:
class BertForSequenceClassification(nn.Module):
    def __init__(self):
        super(BertForSequenceClassification, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size*2, 1)
        self.sigmoid = nn.Sigmoid()
        nn.init.xavier_normal_(self.classifier.weight)

    def forward_once(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        p, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        print("p", p.shape, pooled_output.shape)
        pooled_output = self.dropout(pooled_output)
        return pooled_output

    def forward(self, input_ids1, input_ids2):
        output1 = self.forward_once(input_ids1, token_type_ids=None, attention_mask=None, labels=None)
        print("output1", output1.shape)
        output2 = self.forward_once(input_ids2, token_type_ids=None, attention_mask=None, labels=None)
        print("output2", output2.shape)
        out = torch.cat((output1, output2), 1)
        print("out", out.shape)
        logits = self.classifier(out)
        probab = self.sigmoid(logits)
        return probab

    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False

    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [ ]:
class custom_dataset(Dataset):
    def __init__(self, statement, metadata, label, transform=None, max_len_statement=64, max_len_metadata=32):
        self.statement = statement
        self.metadata = metadata
        self.label = label
        self.transform = transform
        self.max_len_statement = max_len_statement
        self.max_len_metadata = max_len_metadata

    def __getitem__(self,index):
        tokenized_statement = tokenizer.tokenize(self.statement[index])
        statement_id = tokenizer.convert_tokens_to_ids(tokenized_statement)
        statement_id = statement_id[:min(len(statement_id), self.max_len_statement)]
        
        tokenized_metadata = tokenizer.tokenize(self.metadata[index])
        metadata_id = tokenizer.convert_tokens_to_ids(tokenized_metadata)
        metadata_id = metadata_id[:min(len(metadata_id), self.max_len_metadata)]

        label = self.label[index]

        return torch.tensor(statement_id), torch.tensor(metadata_id), label

    def __len__(self):
        return len(self.statement)

In [ ]:
def padding_function(batch):
  statement_batch = []
  metadata_batch = []
  y_batch = []
  for b in batch:
    s, m, y = b
    statement_batch.append(s)
    metadata_batch.append(m)
    y_batch.append(y)
  statement_batch = pad_sequence(statement_batch, batch_first=True, padding_value=0)
  metadata_batch = pad_sequence(metadata_batch, batch_first=True, padding_value=0)
  y_batch = torch.Tensor(y_batch)
  return statement_batch, metadata_batch, y_batch

In [ ]:
train_dataset = custom_dataset(X_train_statement, X_train_metadata, y_train)
val_dataset = custom_dataset(X_val_statement, X_val_metadata, y_val)
test_dataset = custom_dataset(X_test_statement, X_test_metadata, y_test)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True, collate_fn=padding_function)
val_loader = DataLoader(val_dataset, batch_size = 16, shuffle=False, collate_fn=padding_function)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle=False, collate_fn=padding_function)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model = BertForSequenceClassification()
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

In [ ]:
dataloaders_dict = {
    "train": train_loader,
    "val": val_loader,
    "test": test_loader
}

dataset_sizes = {
    "train": len(train_dataset),
    "val": len(val_dataset),
    "test": len(test_dataset)
}

In [ ]:
num_epochs = 25

lrlast = .0001
lrmain = .00001
optim1 = torch.optim.Adam(
    [
        {"params":model.bert.parameters(),"lr": lrmain},
        {"params":model.classifier.parameters(), "lr": lrlast},

   ])
optimizer = optim1

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
criterion = nn.BCELoss().to(device)
model = model.to(device)

In [ ]:
train_acc = train_loss = val_acc = val_loss = []
best_loss = 100
best_acc = 0

In [ ]:
for epoch in range(num_epochs):
        epoch_start = time.time()
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
                # model.freeze_bert_encoder()
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            fakeness_corrects = 0


            # Iterate over data.
            for (statements, metadatas, labels) in tqdm(dataloaders_dict[phase]):
                statements = statements.to(device)
                metadatas = metadatas.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    #print(inputs)
                    outputs = model(statements, metadatas)

                    # outputs = F.softmax(outputs,dim=1)

                    # loss = criterion(outputs, torch.max(fakeness.float(), 1)[1])
                    loss = criterion(outputs.view(outputs.size(0)), labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':

                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * statements.size(0)


                fakeness_corrects += torch.sum((outputs.view(-1) >= 0.5) == (labels >= 0.5))


            epoch_loss = running_loss / dataset_sizes[phase]


            fakeness_acc = fakeness_corrects.double() / dataset_sizes[phase]

            print('{} total loss: {:.4f} '.format(phase,epoch_loss ))
            print('{} fakeness_acc: {:.4f}'.format(
                phase, fakeness_acc))

            # Saving training acc and loss for each epoch
            fakeness_acc1 = fakeness_acc.data
            fakeness_acc1 = fakeness_acc1.cpu()
            fakeness_acc1 = fakeness_acc1.numpy()
            train_acc.append(fakeness_acc1)

            #epoch_loss1 = epoch_loss.data
            #epoch_loss1 = epoch_loss1.cpu()
            #epoch_loss1 = epoch_loss1.numpy()
            train_loss.append(epoch_loss)

            if phase == 'val' and fakeness_acc > best_acc:
                print('Saving with accuracy of {}'.format(fakeness_acc),
                      'improved over previous {}'.format(best_acc))
                best_acc = fakeness_acc

                # Saving val acc and loss for each epoch
                fakeness_acc1 = fakeness_acc.data
                fakeness_acc1 = fakeness_acc1.cpu()
                fakeness_acc1 = fakeness_acc1.numpy()
                val_acc.append(fakeness_acc1)

                #epoch_loss1 = epoch_loss.data
                #epoch_loss1 = epoch_loss1.cpu()
                #epoch_loss1 = epoch_loss1.numpy()
                val_loss.append(epoch_loss)

                # best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'bert_model_test_noFC1_triBERT_binary_focalloss.pth')

        print('Time taken for epoch'+ str(epoch+1)+ ' is ' + str((time.time() - epoch_start)/60) + ' minutes')
        print()

/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1/25
----------


  0%|          | 0/640 [00:00<?, ?it/s]

p torch.Size([16, 52, 768]) torch.Size([16, 768])
output1 torch.Size([16, 768])
p torch.Size([16, 32, 768]) torch.Size([16, 768])
output2 torch.Size([16, 768])
out torch.Size([16, 1536])


  0%|          | 2/640 [00:00<03:05,  3.44it/s]

...

p torch.Size([16, 40, 768]) torch.Size([16, 768])
output1 torch.Size([16, 768])
p torch.Size([16, 32, 768]) torch.Size([16, 768])
output2 torch.Size([16, 768])
out torch.Size([16, 1536])


  2%|▏         | 11/640 [00:02<02:41,  3.90it/s]

p torch.Size([16, 26, 768]) torch.Size([16, 768])
output1 torch.Size([16, 768])
p torch.Size([16, 30, 768]) torch.Size([16, 768])
output2 torch.Size([16, 768])
out torch.Size([16, 1536])


  2%|▏         | 11/640 [00:03<02:53,  3.63it/s]


KeyboardInterrupt: ignored